# Lab 3: Text Generation with an N-Gram Language Model

In this lab, you will implement two text generation strategies covered in lecture: Greedy Search and Sampling.

Here's a revision of the n-gram language model implementation from the previous lab. It now includes a `get_prob_dist()` function, which returns the probabilities of all tokens given the context.

Look over the implementation to be sure that you understand it.

In [1]:
import pickle
BOS = '<BOS>'
EOS = '<EOS>'
OOV = '<OOV>'
class NGramLM:
    def __init__(self, path, smoothing=0.001, verbose=False):
        with open(path, 'rb') as fin:
            data = pickle.load(fin)
        self.n = data['n']
        self.V = set(data['V'])
        self.model = data['model']
        self.smoothing = smoothing
        self.verbose = verbose

    def get_prob_dist(self, context):
        # Take only the n-1 most recent context (Markov Assumption)
        context = tuple(context[-self.n+1:])
        # Add <BOS> tokens if the context is too short, i.e., it's at the start of the sequence
        while len(context) < (self.n-1):
            context = (BOS,) + context
        # Handle words that were not encountered during the training by replacing them with a special <OOV> token
        context = tuple((c if c in self.V else OOV) for c in context)
        if context in self.model:
            # Compute the probability distribution using a Maximum Likelihood Estimation and Laplace Smoothing
            norm = sum(self.model[context].values()) + self.smoothing * len(self.V)
            prob_dist = {k: (c + self.smoothing) / norm for k, c in self.model[context].items()}
            for word in self.V - prob_dist.keys():
                prob_dist[word] = self.smoothing / norm
        else:
            # Simplified formula if we never encountered this context; the probability of all tokens is uniform
            prob = 1 / len(self.V)
            prob_dist = {k: prob for k in self.V}
        prob_dist = dict(sorted(prob_dist.items(), key=lambda x: (-x[1], x[0])))
        return prob_dist

In [2]:
# Load pre-built n-gram languae models
model_unigram = NGramLM('arthur-conan-doyle.tok.train.n1.pkl')
model_bigram = NGramLM('arthur-conan-doyle.tok.train.n2.pkl')
model_trigram = NGramLM('arthur-conan-doyle.tok.train.n3.pkl')
model_4gram = NGramLM('arthur-conan-doyle.tok.train.n4.pkl')
model_5gram = NGramLM('arthur-conan-doyle.tok.train.n5.pkl')

Let's take a look at some of the probability distributions.

Are they reasonable?

In [6]:
model_bigram.get_prob_dist(['elemental'])

{'forces': 0.10647581546320438,
 '!': 5.3211302080561916e-05,
 '"': 5.3211302080561916e-05,
 '&': 5.3211302080561916e-05,
 "'": 5.3211302080561916e-05,
 "'S": 5.3211302080561916e-05,
 "'d": 5.3211302080561916e-05,
 "'em": 5.3211302080561916e-05,
 "'ll": 5.3211302080561916e-05,
 "'m": 5.3211302080561916e-05,
 "'re": 5.3211302080561916e-05,
 "'s": 5.3211302080561916e-05,
 "'ve": 5.3211302080561916e-05,
 '(': 5.3211302080561916e-05,
 ')': 5.3211302080561916e-05,
 '):': 5.3211302080561916e-05,
 '+': 5.3211302080561916e-05,
 ',': 5.3211302080561916e-05,
 '-': 5.3211302080561916e-05,
 '--': 5.3211302080561916e-05,
 '-------------+': 5.3211302080561916e-05,
 '-----------------------------------------------------+': 5.3211302080561916e-05,
 '-I': 5.3211302080561916e-05,
 '-just': 5.3211302080561916e-05,
 '.': 5.3211302080561916e-05,
 '..': 5.3211302080561916e-05,
 '...': 5.3211302080561916e-05,
 '....': 5.3211302080561916e-05,
 '.ERE': 5.3211302080561916e-05,
 '.M': 5.3211302080561916e-05,
 '.

In [4]:
model_bigram.get_prob_dist(['.'])

{'"': 0.23905809267120004,
 '<EOS>': 0.12003662173077437,
 'I': 0.08395797400078746,
 'He': 0.043522818983357844,
 'It': 0.04326904185981749,
 'The': 0.03899712694688821,
 "'": 0.02368590715995356,
 'But': 0.022036355856941265,
 'You': 0.021063543550036576,
 'There': 0.018991030374457027,
 'We': 0.014634523087014295,
 'Then': 0.012519713724178018,
 'She': 0.012139048038867487,
 'A': 0.01040490436134174,
 'If': 0.01032031198682829,
 'His': 0.009812757739747583,
 'And': 0.009347499679923602,
 'This': 0.00879764924558617,
 'That': 0.008543872122045817,
 'In': 0.008290094998505464,
 'As': 0.008205502623992013,
 'What': 0.007782540751424758,
 'Now': 0.007444171253370953,
 'Holmes': 0.0066828398827498935,
 'When': 0.006640543695493168,
 'My': 0.006217581822925912,
 'They': 0.005921508512128833,
 'At': 0.005710027575845205,
 'No': 0.005117880954251048,
 'On': 0.004779511456197243,
 'For': 0.004271957209116537,
 'Well': 0.004187364834603086,
 'So': 0.0041450686473463606,
 'One': 0.003426033463

In [24]:
model_trigram.get_prob_dist(["no",",","I"])

{'am': 0.07304242305524684,
 'have': 0.06723229633638311,
 'think': 0.051461952385181524,
 'should': 0.043161771358233327,
 'was': 0.03984169894745405,
 'can': 0.029051463612421388,
 'do': 0.028221445509726568,
 'will': 0.028221445509726568,
 'had': 0.025731391201642112,
 'could': 0.022411318790862833,
 'suppose': 0.022411318790862833,
 'shall': 0.020751282585473194,
 'saw': 0.01909124638008355,
 'found': 0.01660119207199909,
 "'ll": 0.014941155866609454,
 'see': 0.014111137763914634,
 'would': 0.014111137763914634,
 'believe': 0.013281119661219814,
 'fancy': 0.013281119661219814,
 'know': 0.013281119661219814,
 'must': 0.013281119661219814,
 'did': 0.011621083455830173,
 'knew': 0.011621083455830173,
 'may': 0.011621083455830173,
 'presume': 0.010791065353135352,
 'understand': 0.010791065353135352,
 'thought': 0.009961047250440532,
 "'m": 0.009131029147745712,
 'fear': 0.009131029147745712,
 'say': 0.008301011045050892,
 'hope': 0.007470992942356073,
 'never': 0.006640974839661253,
 

Great, now we have all the tools we need to start generating text!

We'll start with a simple greedy generation approach. Your task is to implement greedy generation below.

Note: we have a `max_length` parameter to be sure that the generation process doesn't go on forever. You can stop when your sequence either reaches an `<EOS>` token or is the maximum length.

In [33]:
from typing import List
def greedy_generation(model: NGramLM, context: List[str], max_length: int = 100) -> List[str]:
    
    generated_sequence = context[:]
    while len(generated_sequence) < max_length:
            prob_dist = model.get_prob_dist(generated_sequence)
            next_word = max(prob_dist, key=prob_dist.get)
            if next_word == EOS:
                break
            generated_sequence.append(next_word)
    return generated_sequence

greedy_generation(model_4gram, ["elemental",",","my","dear"])

['elemental',
 ',',
 'my',
 'dear',
 'Watson',
 ',',
 '"',
 'said',
 'he',
 '.',
 '"',
 'I',
 'am',
 'afraid',
 'that',
 'I',
 'must',
 'have',
 'a',
 'look',
 'at',
 'the',
 'matter',
 '.',
 '"']

Great! How does the generation look? Feel free to try out a several samples below.

Is it deterministic? Are the generated sequences interesting?

Consider trying different model types. What are the different qualities that you see from the unigram, bigram, trigram, 4-gram, and 5-gram models?

In [37]:
from typing import List
def greedy_generation(model: NGramLM, context: List[str], max_length: int = 100) -> List[str]:
    
    generated_sequence = context[:]
    while len(generated_sequence) < max_length:
            prob_dist = model.get_prob_dist(generated_sequence)
            next_word = max(prob_dist, key=prob_dist.get)
            if next_word == EOS:
                break
            generated_sequence.append(next_word)
    return generated_sequence

greedy_generation(model_trigram, ["elemental",",","my","dear"])

['elemental', ',', 'my', 'dear', 'Watson', ',', '"', 'said', 'he', '.', '"']

Now it's time to implement sampling.

We now include a `topk` argument. This reduces the candidate set of probabilities down to only the `topk` highest-probability items. This helps reduce the chance of generating highly unlikely sequences.

Note: consider using [`random.choices`](https://docs.python.org/3/library/random.html#random.choices) to help in sampling.

In [40]:
from typing import List
import random
def sampling_generation(model: NGramLM, context: List[str], max_length: int = 100, topk=10) -> List[str]:
    generated_sequence = context[:]
    while len(generated_sequence) < max_length:
        prob_dist = model.get_prob_dist(generated_sequence)
        # Reduce the candidate set to only the topk highest-probability items
        topk_probs = sorted(prob_dist.items(), key=lambda x: x[1], reverse=True)[:topk]
        topk_words = [word for word, prob in topk_probs]
        # Use random.choices to sample the next word from the topk words
        next_word = random.choices(topk_words, weights=[prob for word, prob in topk_probs], k=1)[0]
        if next_word == EOS:
            break
        generated_sequence.append(next_word)
    return generated_sequence

sampling_generation(model_trigram, ['""', 'My', 'dear', 'Watson'])

['""',
 'My',
 'dear',
 'Watson',
 ',',
 'you',
 'will',
 'see',
 'a',
 'beautiful',
 'moonlight',
 'night',
 ',',
 'and',
 'then',
 'suddenly',
 'realizing',
 'how',
 'he',
 'found',
 'an',
 'extraordinary',
 'thing',
 ',',
 '"',
 'said',
 'our',
 'learned',
 'guide',
 '.']

Now qualitatively compare your sampling generation with the greedy generation.

What do you notice about the generated sequences? How do models of different sizes behave? What is the effect of `topk`?

In [ ]:
# your code here

## Optional Extras:
 - Try implementing a beam search strategy. Does it tend to lead to qualitatively better results than the other two approaches?
 - What strategy might you take to efficiently find the most likely sequence for an n-gram language model?